# Colab-VSGAN

My repo: [styler00dollar/VSGAN-tensorrt-docker](https://github.com/styler00dollar/VSGAN-tensorrt-docker/)

I am not sure why, but the compact model does not seem to work.

```
[lrc @ 0x55c14252c000] Format lrc detected only with low score of 5, misdetection possible!
```
The other stuff seems to work.

In [ ]:
!nvidia-smi

In [ ]:
!nvcc --version

In [ ]:
# check python version, for me its currently 3.7.12
!python --version

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)



You need to get 2 files. Currently, the cuda version inside Colab is 11.1, that's why you need to get:
```
nv-tensorrt-repo-ubuntu1804-cuda11.4-trt8.2.3.0-ga-20220113_1-1_amd64.deb

(TensorRT 8.2 GA Update 2 for Ubuntu 18.04 and CUDA 11.0, 11.1, 11.2, 11.3, 11.4 and 11.5 DEB local repo Package)

and

TensorRT-8.2.3.0.Linux.x86_64-gnu.cuda-11.4.cudnn8.2.tar.gz/python/tensorrt-8.2.3.0-cp37-none-linux_x86_64.whl

(TensorRT 8.2 GA Update 2 for Linux x86_64 and CUDA 11.0, 11.1, 11.2, 11.3, 11.4 and 11.5 TAR Package)

```
You can download these files [here](https://developer.nvidia.com/nvidia-tensorrt-download). Warning: You need an account (which can be created for free). Put them into `gdrive/TensorRT`.

In [ ]:
#@title install
import os
os.environ["os1"]="ubuntu1804"
os.environ["tag"]= "cuda11.4-trt8.2.3.0-ga-20220113" #@param
os.environ["version"]= "8.2.3-1+cuda11.4" #@param
data_path = '/content/drive/MyDrive/TensorRT/' #@param
os.chdir(data_path)
!sudo dpkg -i nv-tensorrt-repo-${os1}-${tag}_1-1_amd64.deb
!sudo apt-key add /var/nv-tensorrt-repo-${tag}/7fa2af80.pub
!sudo apt-get update
!sudo apt-get install libnvinfer8=${version} libnvonnxparsers8=${version} libnvparsers8=${version} libnvinfer-plugin8=${version} libnvinfer-dev=${version} libnvonnxparsers-dev=${version} libnvparsers-dev=${version} libnvinfer-plugin-dev=${version} python3-libnvinfer=${version}
!sudo apt-mark hold libnvinfer8 libnvonnxparsers8 libnvparsers8 libnvinfer-plugin8 libnvinfer-dev libnvonnxparsers-dev libnvparsers-dev libnvinfer-plugin-dev python3-libnvinfer
!sudo apt-get install tensorrt=8.2.3.0-1+cuda11.4
!sudo apt-get install python3-libnvinfer-dev=${version}

`Runtime -> Restart runtime`

In [ ]:
# install the whl with the matching python version
!pip install "/content/drive/MyDrive/TensorRT/tensorrt-8.2.3.0-cp37-none-linux_x86_64.whl"

`Runtime -> Restart runtime`

Install everything. Avoid compiling mmcv if you can, since the total install time will be long.

In [ ]:
#@title install (be patient, ~12min)
%cd /content/
!sudo rm -rf /workspace
!mkdir /workspace/tensorrt
%cd /workspace/tensorrt

# installing vapoursynth
!apt install ffmpeg autoconf libtool yasm python3.9 python3.9-venv python3.9-dev ffmsindex libffms2-4 libffms2-dev -y
!apt --fix-broken install
!git clone https://github.com/sekrit-twc/zimg.git && cd zimg && ./autogen.sh && ./configure && make -j4 && make install && cd .. && rm -rf zimg
!pip install Cython -U --force-reinstall
!wget https://github.com/vapoursynth/vapoursynth/archive/refs/tags/R57.zip -P /workspace/tensorrt
%cd /workspace/tensorrt/
!7z x /workspace/tensorrt/R57.zip
%cd /workspace/tensorrt/vapoursynth-R57
!./autogen.sh && ./configure && make && make install && cd .. && ldconfig
!ln -s /usr/local/lib/python3.7/site-packages/vapoursynth.so /usr/lib/python3.7/lib-dynload/vapoursynth.so
!pip install vapoursynth

# cupy / pycuda
!curl https://colab.chainer.org/install | sh -
!pip install pycuda

# pytorch tensorrt
!pip install https://github.com/NVIDIA/Torch-TensorRT/releases/download/v1.0.0/torch_tensorrt-1.0.0-cp37-cp37m-linux_x86_64.whl

# onnx
!pip install onnx onnxruntime onnxruntime-gpu

# installing onnx tensorrt with a workaround, error with import otherwise
%cd /workspace/tensorrt
!git clone https://github.com/onnx/onnx-tensorrt
%cd onnx-tensorrt
!python3 setup.py install

%cd /workspace
# downloading models
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.3.0/RealESRGANv2-animevideo-xsx2.pth
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.3.0/RealESRGANv2-animevideo-xsx4.pth
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth
# fatal anime
!wget https://de-next.owncube.com/index.php/s/x99pKzS7TNaErrC/download -O 4x_fatal_Anime_500000_G.pth
# rvp1
!pip install gdown && gdown --id 1IJe6WLvT43iwl-3J6ectgnjas5mjnQ51
# sepconv
!wget http://content.sniklaus.com/resepconv/network-paper.pytorch -O sepconv.pth
# EGVSR
!wget https://github.com/Thmen/EGVSR/raw/master/pretrained_models/EGVSR_iter420000.pth
# rife4 (fixed rife4.0 model)
!gdown --id 1UzCbpjxWJsfiDjoc7wuzq3K0RCf5Oxr3
# RealBasicVSR_x4
!gdown --id 1OYR1J2GXE90Zu2gVU5xc0t0P_UmKH7ID

# optional, rvp uses it to convert colorspace
!pip install kornia
# image read/write for image inference
!pip install opencv-python

# vs plugings from others
# https://github.com/HolyWu/vs-swinir
!pip install --upgrade vsswinir && python -m vsswinir

# pytorch hotfix
!pip uninstall torch -y
!pip uninstall torch -y
!pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

# onnx gpu hotfix
!pip install onnxruntime-gpu --force-reinstall -U

# jpg turbo
!sudo apt-get install -y libturbojpeg 
!pip install PyTurboJPEG

In [ ]:
#@title (optional) compile mmcv (basicvsrpp/RealBasicVSR)
#@markdown Compiling mmcv takes a while, only do this if you need it
# https://github.com/HolyWu/vs-basicvsrpp
!pip install --upgrade vsbasicvsrpp && python -m vsbasicvsrpp
# dependencies for RealBasicVSR_x4
# mmedit
!git clone https://github.com/open-mmlab/mmediting.git && cd mmediting && pip install -v -e .
# RealBasicVSR_x4 will download this
!wget "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" -P /root/.cache/torch/hub/checkpoints/

`Runtime -> Restart runtime`

Dependencies are installed now.

In [ ]:
%cd /workspace/tensorrt
!git clone https://github.com/styler00dollar/VSGAN-tensorrt-docker
%cd /workspace/tensorrt/VSGAN-tensorrt-docker

In [ ]:
#@title inference.py
%%writefile /workspace/tensorrt/VSGAN-tensorrt-docker/inference.py
import sys
sys.path.append('/workspace/tensorrt/VSGAN-tensorrt-docker/')
import vapoursynth as vs
from src.esrgan import ESRGAN_inference # esrgan and realesrgan
from src.SRVGGNetCompact import SRVGGNetCompactRealESRGAN # realesrgan anime video
from src.vfi_model import video_model # any vfi model, in this case rvp1 as demonstration
from src.sepconv_enhanced import sepconv_model # uses cupy, no tensorrt
from src.rife import RIFE # tensorrt not possible
from vsswinir import SwinIR # https://github.com/HolyWu/vs-swinir # currently not tensorrt, didn't try
from src.egvsr import egvsr_model # currently not tensorrt
#from vsbasicvsrpp import BasicVSRPP
#from src.realbasicvsr import realbasicvsr_model

core = vs.core
vs_api_below4 = vs.__api_version__.api_major < 4
core = vs.core
core.num_threads = 16 # can influence ram usage
# only needed if you are inside docker
core.std.LoadPlugin(path='/usr/lib/x86_64-linux-gnu/libffms2.so')

# cfr video
clip = core.ffms2.Source(source='480.mkv')
# vfr video (untested)
#clip = core.ffms2.Source(source='input.mkv', fpsnum = 24000, fpsden = 1001)
###############################################
# COLORSPACE
###############################################
# convert colorspace
clip = vs.core.resize.Bicubic(clip, format=vs.RGBS, matrix_in_s='709')
# convert colorspace + resizing
#clip = vs.core.resize.Bicubic(clip, width=848, height=480, format=vs.RGBS, matrix_in_s='709')

###############################################

# these demos work out of the box because docker also downloads the needed models, if you want other models, just add them
# you can combine everything however you want

###############################################
# MODELS (CUDA)
###############################################
# sepconv
#clip = sepconv_model(clip)
# RIFE4
clip = RIFE(clip, multi = 2, scale = 1.0, fp16 = True, fastmode = False, ensemble = True)
# VFI example for jit models
#clip = video_model(clip, fp16=False, model_path="/workspace/rvpV1_105661_G.pt")
# SwinIR
#clip = SwinIR(clip, task="lightweight_sr", scale=2)
# ESRGAN / RealESRGAN
#clip = ESRGAN_inference(clip=clip, model_path="/workspace/4x_fatal_Anime_500000_G.pth", tile_x=400, tile_y=400, tile_pad=10, fp16=False)
#clip = ESRGAN_inference(clip=clip, model_path="/workspace/RealESRGAN_x4plus_anime_6B.pth", tile_x=480, tile_y=480, tile_pad=16, fp16=False)
# RealESRGAN Anime Video example
# backends: tenssorrt, cuda, onnx, quantized_onnx
#clip = SRVGGNetCompactRealESRGAN(clip, scale=2, fp16=True, backend = "cuda")
# EGVSR
#clip = egvsr_model(clip, interval=15)
# BasicVSR++
# 0 = REDS, 1 = Vimeo-90K (BI), 2 = Vimeo-90K (BD), 3 = NTIRE 2021 - Track 1, 4 = NTIRE 2021 - Track 2, 5 = NTIRE 2021 - Track 3
#clip = BasicVSRPP(clip, model = 1, interval = 30, tile_x = 0, tile_y = 0, tile_pad = 16, device_type = 'cuda', device_index = 0, fp16 = False, cpu_cache = False)
# RealBasicVSR
#clip = realbasicvsr_model(clip, interval=15, fp16=True)

###############################################
# [NOT IN DOCKER] MODELS (NCNN)
# Only recommended for AMD GPUS, further instructions in README
###############################################
#from src.SRVGGNetCompact_ncnn import SRVGGNetCompactRealESRGAN_ncnn

# Rife ncnn
# 0 = rife-v3.1, 1 = rife-v3.0, 2 = rife-v2.4, 3 = rife-v2, 4 = rife-anime
#clip = core.rife.RIFE(clip, model=0, gpu_id=0, gpu_thread=2, tta=False, uhd=False, sc=False, list_gpu=False)
# RealESRGAN example
#clip = SRVGGNetCompactRealESRGAN_ncnn(clip, gpuid=0, model="models-DF2K", tta_mode=False, scale = 2, tilesize=0, param_path = None, bin_path = None)
# Waifu2x
# 0 = upconv_7_anime_style_art_rgb, 1 = upconv_7_photo, 2 = cunet (For 2D artwork. Slow, but better quality.)
#clip = core.w2xnvk.Waifu2x(clip, noise=0, scale=2, model=0, tile_size=0, gpu_id=0, gpu_thread=0, precision=16)

###############################################
# OUTPUT
###############################################
clip = vs.core.resize.Bicubic(clip, format=vs.YUV420P8, matrix_s="709")
clip.set_output()


In [ ]:
%cd /workspace/tensorrt/VSGAN-tensorrt-docker/
!vspipe -c y4m inference.py - | ffmpeg -i pipe: /workspace/tensorrt/VSGAN-tensorrt-docker/example.mkv